# Experimento mestrado 1
## Executando na pasta principal

In [1]:
import os
import numpy as np
os.chdir('../../../') #Executando na pasta Visual_Memory
import sys
sys.path.append('./include')
sys.path.append('./src')

## Copilando arquivos para teste

In [2]:
if os.system("cd Workbench/ && ./convertendo_all.sh && cd ..") != 0:
    print "Erro de copilação"
    sys.exit(0)

## Criando configurações padroes e arquivo de execução

In [3]:
ERRO = 1

os.system('sed -i \'s/time.time( )/self._bkb.read_float("VISION_RB20_TAG")/g\' ./include/Landmark.py')
os.system('sed -i \'s/time.time( )/self._bkb.read_float("VISION_RB20_TAG")/g\' ./include/KalmanFilter.py')
os.system('sed -i \'s/time.time( )/self._bkb.read_float("VISION_RB20_TAG")/g\' ./src/Behavior.py')
os.system('sed -i \'/time.sleep(delta)/d\' ./src/Behavior.py')
os.system('sed -i \'s/if delta > 0:/self._bkb.write_float("VISION_RB21_TAG", 0)\\n                while self._bkb.read_float("VISION_RB21_TAG") == 0:\\n                    time.sleep(0.5)/g\' ./src/Behavior.py')

text = open("./Data/config.ini", "w")
text.write("[KALMAN FILTER - ROBOTS]\nvision_error = "+ str(ERRO) +"\n\n[KALMAN FILTER - LANDMARKS]\nvision_error = "+ str(ERRO))
text.close()

text = open("./run.sh", "w")
# text.write("#!/bin/bash\n\npython visualMemory.py --nr 3 --p 2000 --d --sd")
text.write("#!/bin/bash\n\npython visualMemory.py --nr 3 --p 2000 --sd")
text.close()

In [4]:
from Blackboard import *
import random
import time
from copy import copy

In [5]:
bkb = Blackboard( )

In [6]:
bkb.write_float('VISION_RB20_TAG', 0) # tempo no now
bkb.write_float('VISION_RB21_TAG', 0) # Parando visual 

# Zerando robôs
bkb.write_float('VISION_RB01_TAG', 0)
bkb.write_float('VISION_RB02_TAG', 0)

bkb.write_float('VISION_LAND_TAG', 0)

bkb.write_float("VISUAL_MEMORY_LAND_LOC", 0)
bkb.write_float("VISUAL_MEMORY_LAND_X", 0)
bkb.write_float("VISUAL_MEMORY_LAND_Y", 0)

bkb.write_float("VISUAL_MEMORY_AL_01_LOC", 0)
bkb.write_float("VISUAL_MEMORY_AL_01_X", 0)
bkb.write_float("VISUAL_MEMORY_AL_01_Y", 0)
bkb.write_float("VISUAL_MEMORY_AL_01_MAX_VEL", 0)

bkb.write_float("VISUAL_MEMORY_OP_01_LOC", 0)
bkb.write_float("VISUAL_MEMORY_OP_01_X", 0)
bkb.write_float("VISUAL_MEMORY_OP_01_Y", 0)
bkb.write_float("VISUAL_MEMORY_OP_01_MAX_VEL", 0)

## Executando VisualMemory e removendo arquivo de execução

In [7]:
os.system("chmod 777 run.sh")
os.system("kill $(pidof -x ./run.sh)")
os.system("screen -d -m -S 'VisualMemory' './run.sh'")
os.system("gnome-terminal -x sh -c 'screen -R VisualMemory'")
os.system("rm run.sh")

0

In [ ]:
FALHA = 100
FALHACOR = 20
# vx = 2; vy = 1; vax = 5; vay = -1; vox = -3; voy = -1.5; ω = 1°
listaLand = [
    {"tag": 1, "pos": [+6.00, +3.00], "movement": 1, "time": 0.0},
    {"tag": 1, "pos": [+5.03, +2.45], "movement": 1, "time": 0.5},
    {"tag": 1, "pos": [+4.05, +1.89], "movement": 1, "time": 1.0},
    {"tag": 1, "pos": [+3.08, +1.34], "movement": 1, "time": 1.5},
    {"tag": 1, "pos": [+2.10, +0.79], "movement": 1, "time": 2.0},
    {"tag": 1, "pos": [+1.13, +0.24], "movement": 1, "time": 2.5},
    {"tag": 1, "pos": [+0.15, -0.32], "movement": 1, "time": 3.0},
    {"tag": 1, "pos": [-0.83, -0.87], "movement": 1, "time": 3.5},
    {"tag": 1, "pos": [-1.81, -1.43], "movement": 1, "time": 4.0},
    {"tag": 1, "pos": [-2.78, -1.98], "movement": 1, "time": 4.5},
    {"tag": 1, "pos": [-3.76, -2.53], "movement": 1, "time": 5.0},
    {"tag": 1, "pos": [-4.74, -3.09], "movement": 1, "time": 5.5},
    {"tag": 1, "pos": [-5.72, -3.64], "movement": 1, "time": 6.0},
    {"tag": 1, "pos": [-6.70, -4.20], "movement": 1, "time": 6.5},
    {"tag": 1, "pos": [-7.68, -4.75], "movement": 1, "time": 7.0},
    {"tag": 1, "pos": [-8.66, -5.31], "movement": 1, "time": 7.5},
    {"tag": 1, "pos": [-9.64, -5.86], "movement": 1, "time": 8.0},
]

listaOponente = [
    {"tag": 1, "pos": [+7.00, +0.00], "movement": 1, "time": 0.0},
    {"tag": 1, "pos": [+4.49, -1.30], "movement": 1, "time": 0.5},
    {"tag": 1, "pos": [+1.97, -2.57], "movement": 1, "time": 1.0},
    {"tag": 1, "pos": [-0.56, -3.81], "movement": 1, "time": 1.5},
    {"tag": 1, "pos": [-3.11, -5.03], "movement": 1, "time": 2.0},
    {"tag": 1, "pos": [-5.66, -6.22], "movement": 1, "time": 2.5},
    {"tag": 1, "pos": [-8.23, -7.39], "movement": 1, "time": 3.0},
    {"tag": 1, "pos": [-10.8, -8.53], "movement": 1, "time": 3.5},
    {"tag": 1, "pos": [-13.4, -9.64], "movement": 1, "time": 4.0},
    {"tag": 1, "pos": [-16.0, -10.7], "movement": 1, "time": 4.5},
    {"tag": 1, "pos": [-18.6, -11.8], "movement": 1, "time": 5.0},
    {"tag": 1, "pos": [-21.3, -12.8], "movement": 1, "time": 5.5},
    {"tag": 1, "pos": [-23.9, -13.8], "movement": 1, "time": 6.0},
    {"tag": 1, "pos": [-26.5, -14.7], "movement": 1, "time": 6.5},
    {"tag": 1, "pos": [-29.1, -15.7], "movement": 1, "time": 7.0},
    {"tag": 1, "pos": [-31.8, -16.6], "movement": 1, "time": 7.5},
    {"tag": 1, "pos": [-34.5, -17.5], "movement": 1, "time": 8.0},
]

listaAliado = [
    {"tag": 3, "pos": [+6.00, +0.00], "movement": 1, "time": 0.0},
    {"tag": 3, "pos": [+4.00, -1.04], "movement": 1, "time": 0.5},
    {"tag": 3, "pos": [+1.98, -1.97], "movement": 1, "time": 1.0},
    {"tag": 3, "pos": [-0.04, -3.00], "movement": 1, "time": 1.5},
    {"tag": 3, "pos": [-2.07, -4.07], "movement": 1, "time": 2.0},
    {"tag": 3, "pos": [-4.11, -5.04], "movement": 1, "time": 2.5},
    {"tag": 3, "pos": [-6.16, -6.00], "movement": 1, "time": 3.0},
    {"tag": 3, "pos": [-8.21, -6.93], "movement": 1, "time": 3.5},
    {"tag": 3, "pos": [-10.3, -7.85], "movement": 1, "time": 4.0},
    {"tag": 3, "pos": [-12.3, -8.75], "movement": 1, "time": 4.5},
    {"tag": 3, "pos": [-14.4, -9.63], "movement": 1, "time": 5.0},
    {"tag": 3, "pos": [-16.5, -10.5], "movement": 1, "time": 5.5},
    {"tag": 3, "pos": [-18.6, -11.3], "movement": 1, "time": 6.0},
    {"tag": 3, "pos": [-20.7, -12.2], "movement": 1, "time": 6.5},
    {"tag": 3, "pos": [-22.8, -13.0], "movement": 1, "time": 7.0},
    {"tag": 3, "pos": [-24.9, -13.8], "movement": 1, "time": 7.5},
    {"tag": 3, "pos": [-27.0, -14.5], "movement": 1, "time": 8.0},
]


for lista in [listaLand, listaOponente, listaAliado]:
    for data in lista:
        if random.random() <= FALHA/100.0:
            data["pos"][0] += random.uniform(-ERRO, ERRO)
            data["pos"][1] += random.uniform(-ERRO, ERRO)
        else:
            lista.remove(data)

for lista in [listaOponente, listaAliado]:
    for data in lista:
        if random.random() > FALHACOR/100.0:
            data["tag"] = 2

In [ ]:
# raw_input("Pronto para iniciar experimentos")
time.sleep(3)

clistaLand = []
clistaOponente = []
clistaAliado = []

for t in np.arange(0, 8.5, 0.5):
    if listaLand != [] and t == listaLand[0]["time"]:
        dataLand = listaLand.pop(0)
    else:
        dataLand = None
    
    if listaOponente != [] and t == listaOponente[0]["time"]:
        dataOponente = listaOponente.pop(0)
    else:
        dataOponente = None
    
    if listaAliado != [] and t == listaAliado[0]["time"]:
        dataAliado = listaAliado.pop(0)
    else:
        dataAliado = None
    
    print dataLand
    print dataAliado
    print dataOponente
    print '\n'
    clistaLand.append(dataLand)
    clistaAliado.append(dataAliado)
    clistaOponente.append(dataOponente)
    
    # Instante atual no tempo
    bkb.write_float('VISION_RB20_TAG', t + 0.5)
    
    if dataLand != None:
        bkb.write_float('VISION_LAND_X', dataLand['pos'][0])
        bkb.write_float('VISION_LAND_Y', dataLand['pos'][1])
        bkb.write_float('VISION_LAND_TIME', dataLand['time'] + 0.5)        
        bkb.write_float('VISION_LAND_MOV', dataLand['movement'])
        bkb.write_float('VISION_LAND_TAG', dataLand['tag'])
    
    if dataOponente != None:
        bkb.write_float('VISION_RB01_X', dataOponente['pos'][0])
        bkb.write_float('VISION_RB01_Y', dataOponente['pos'][1])
        bkb.write_float('VISION_RB01_TIME', dataOponente['time'] + 0.5)        
        bkb.write_float('VISION_RB01_MOV', dataOponente['movement'])
        bkb.write_float('VISION_RB01_TAG', dataOponente['tag'])
    
    if dataAliado != None:
        bkb.write_float('VISION_RB02_X', dataAliado['pos'][0])
        bkb.write_float('VISION_RB02_Y', dataAliado['pos'][1])
        bkb.write_float('VISION_RB02_TIME', dataAliado['time'] + 0.5)        
        bkb.write_float('VISION_RB02_MOV', dataAliado['movement'])
        bkb.write_float('VISION_RB02_TAG', dataAliado['tag'])
    
    # Iniciando
#     raw_input("Pronto para enviar")
    bkb.write_float('VISION_RB21_TAG', 1)
    
    while bkb.read_float('VISION_RB21_TAG'):
        pass
    
#     raw_input("Finalizado\n")

Pronto para iniciar experimentos
{'time': 0.0, 'tag': 1, 'pos': [6.187692166766968, 2.7864778688024967], 'movement': 1}
{'time': 0.0, 'tag': 2, 'pos': [6.388233204270727, -0.8184588829745294], 'movement': 1}
{'time': 0.0, 'tag': 1, 'pos': [6.573320000076532, -0.945255326634366], 'movement': 1}


Pronto para enviar
Finalizado

{'time': 0.5, 'tag': 1, 'pos': [4.780520827184866, 2.0222206191850973], 'movement': 1}
{'time': 0.5, 'tag': 2, 'pos': [4.0085170551704765, -1.3240803591495833], 'movement': 1}
{'time': 0.5, 'tag': 2, 'pos': [4.241504679412513, -2.1642271079348814], 'movement': 1}


Pronto para enviar


In [ ]:
os.system("kill $(pidof -x ./run.sh)")
dataRob1 = np.load("./Data/Thread-1-Robots.npy")
dataRob2 = np.load("./Data/Thread-2-Robots.npy")
dataLand = np.load("./Data/Land.npy")

In [ ]:
clistaLand = [a for a in clistaLand if a != None]
clistaAliado = [a for a in clistaAliado if a != None]
clistaOponente = [a for a in clistaOponente if a != None]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
DataRob1 = []
DataRob2 = []
DataLand = []
for save, data in zip([DataRob1, DataRob2], [dataRob1, dataRob2]):
    for medida in data:
        dados = []
        for tempo, pos, erro, covariance, time in [medida]:
            a = []
            a.append(tempo)
            a.extend(pos)
            a.extend(erro)
            a.extend(covariance)
            a.append(time)
            for i in xrange(len(a)):
                a[i] = float(a[i])
            save.append(a)

for save, data in zip([DataLand], [dataLand]):
    for medida in data:
        dados = []
        for tempo, pos, erro, covariance in [medida]:
            a = []
            a.append(tempo)
            a.extend(pos)
            a.extend(erro)
            a.extend(covariance)
            for i in xrange(len(a)):
                a[i] = float(a[i])
            save.append(a)

In [ ]:
DataRob1 = pd.DataFrame(
    DataRob1,
    columns=[
        "Tempo",
        "Posição X",
        "Posição Y",
        "Velocidade X",
        "Velocidade Y",
        "Erro PX",
        "Erro PY",
        "Erro VX",
        "Erro VY",
        "Covariância PX",
        "Covariância PY",
        "Covariância VX",
        "Covariância VY",
        "Time",
    ]
)
DataRob1

In [ ]:
DataRob2 = pd.DataFrame(
    DataRob2,
    columns=[
        "Tempo",
        "Posição X",
        "Posição Y",
        "Velocidade X",
        "Velocidade Y",
        "Erro PX",
        "Erro PY",
        "Erro VX",
        "Erro VY",
        "Covariância PX",
        "Covariância PY",
        "Covariância VX",
        "Covariância VY",
        "Time",
    ]
)
DataRob2

In [ ]:
DataLand = pd.DataFrame(
    DataLand,
    columns=[
        "Tempo",
        "Posição X",
        "Posição Y",
        "Velocidade X",
        "Velocidade Y",
        "Aceleração X",
        "Aceleração Y",
        "Erro PX",
        "Erro PY",
        "Erro VX",
        "Erro VY",
        "Erro AX",
        "Erro AY",
        "Covariância PX",
        "Covariância PY",
        "Covariância VX",
        "Covariância VY",
        "Covariância AX",
        "Covariância AY",
    ]
)
DataLand

In [ ]:
listaLand = [
    {"tag": 1, "pos": [+6.00, +3.00], "movement": 1, "time": 0.0},
    {"tag": 1, "pos": [+5.03, +2.45], "movement": 1, "time": 0.5},
    {"tag": 1, "pos": [+4.05, +1.89], "movement": 1, "time": 1.0},
    {"tag": 1, "pos": [+3.08, +1.34], "movement": 1, "time": 1.5},
    {"tag": 1, "pos": [+2.10, +0.79], "movement": 1, "time": 2.0},
    {"tag": 1, "pos": [+1.13, +0.24], "movement": 1, "time": 2.5},
    {"tag": 1, "pos": [+0.15, -0.32], "movement": 1, "time": 3.0},
    {"tag": 1, "pos": [-0.83, -0.87], "movement": 1, "time": 3.5},
    {"tag": 1, "pos": [-1.81, -1.43], "movement": 1, "time": 4.0},
    {"tag": 1, "pos": [-2.78, -1.98], "movement": 1, "time": 4.5},
    {"tag": 1, "pos": [-3.76, -2.53], "movement": 1, "time": 5.0},
    {"tag": 1, "pos": [-4.74, -3.09], "movement": 1, "time": 5.5},
    {"tag": 1, "pos": [-5.72, -3.64], "movement": 1, "time": 6.0},
    {"tag": 1, "pos": [-6.70, -4.20], "movement": 1, "time": 6.5},
    {"tag": 1, "pos": [-7.68, -4.75], "movement": 1, "time": 7.0},
    {"tag": 1, "pos": [-8.66, -5.31], "movement": 1, "time": 7.5},
    {"tag": 1, "pos": [-9.64, -5.86], "movement": 1, "time": 8.0},
]

listaOponente = [
    {"tag": 1, "pos": [+7.00, +0.00], "movement": 1, "time": 0.0},
    {"tag": 1, "pos": [+4.49, -1.30], "movement": 1, "time": 0.5},
    {"tag": 1, "pos": [+1.97, -2.57], "movement": 1, "time": 1.0},
    {"tag": 1, "pos": [-0.56, -3.81], "movement": 1, "time": 1.5},
    {"tag": 1, "pos": [-3.11, -5.03], "movement": 1, "time": 2.0},
    {"tag": 1, "pos": [-5.66, -6.22], "movement": 1, "time": 2.5},
    {"tag": 1, "pos": [-8.23, -7.39], "movement": 1, "time": 3.0},
    {"tag": 1, "pos": [-10.8, -8.53], "movement": 1, "time": 3.5},
    {"tag": 1, "pos": [-13.4, -9.64], "movement": 1, "time": 4.0},
    {"tag": 1, "pos": [-16.0, -10.7], "movement": 1, "time": 4.5},
    {"tag": 1, "pos": [-18.6, -11.8], "movement": 1, "time": 5.0},
    {"tag": 1, "pos": [-21.3, -12.8], "movement": 1, "time": 5.5},
    {"tag": 1, "pos": [-23.9, -13.8], "movement": 1, "time": 6.0},
    {"tag": 1, "pos": [-26.5, -14.7], "movement": 1, "time": 6.5},
    {"tag": 1, "pos": [-29.1, -15.7], "movement": 1, "time": 7.0},
    {"tag": 1, "pos": [-31.8, -16.6], "movement": 1, "time": 7.5},
    {"tag": 1, "pos": [-34.5, -17.5], "movement": 1, "time": 8.0},
]

listaAliado = [
    {"tag": 3, "pos": [+6.00, +0.00], "movement": 1, "time": 0.0},
    {"tag": 3, "pos": [+4.00, -1.04], "movement": 1, "time": 0.5},
    {"tag": 3, "pos": [+1.98, -1.97], "movement": 1, "time": 1.0},
    {"tag": 3, "pos": [-0.04, -3.00], "movement": 1, "time": 1.5},
    {"tag": 3, "pos": [-2.07, -4.07], "movement": 1, "time": 2.0},
    {"tag": 3, "pos": [-4.11, -5.04], "movement": 1, "time": 2.5},
    {"tag": 3, "pos": [-6.16, -6.00], "movement": 1, "time": 3.0},
    {"tag": 3, "pos": [-8.21, -6.93], "movement": 1, "time": 3.5},
    {"tag": 3, "pos": [-10.3, -7.85], "movement": 1, "time": 4.0},
    {"tag": 3, "pos": [-12.3, -8.75], "movement": 1, "time": 4.5},
    {"tag": 3, "pos": [-14.4, -9.63], "movement": 1, "time": 5.0},
    {"tag": 3, "pos": [-16.5, -10.5], "movement": 1, "time": 5.5},
    {"tag": 3, "pos": [-18.6, -11.3], "movement": 1, "time": 6.0},
    {"tag": 3, "pos": [-20.7, -12.2], "movement": 1, "time": 6.5},
    {"tag": 3, "pos": [-22.8, -13.0], "movement": 1, "time": 7.0},
    {"tag": 3, "pos": [-24.9, -13.8], "movement": 1, "time": 7.5},
    {"tag": 3, "pos": [-27.0, -14.5], "movement": 1, "time": 8.0},
]



# Analisando dados de Landmarks

In [ ]:
plt.plot([a["time"]+0.5 for a in clistaLand], [a["pos"][0] for a in clistaLand], "r^", label="Dado enviado")
plt.plot([i["time"]+0.5 for i in listaLand], [i["pos"][0] for i in listaLand], "g", label= "Valor real")
plt.errorbar(DataLand["Tempo"], DataLand["Posição X"], DataLand["Erro PX"], fmt='bx', label= "Pontos calculados")
plt.xlabel("Tempo (s)")
plt.ylabel(u"Posição em X")
plt.ylim([-10, 8])
plt.xlim([0, 9])
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot([a["time"]+0.5 for a in clistaLand], [a["pos"][1] for a in clistaLand], "r^", label="Dado enviado")
plt.plot([i["time"]+0.5 for i in listaLand], [i["pos"][1] for i in listaLand], "g", label= "Valor real")
plt.errorbar(DataLand["Tempo"], DataLand["Posição Y"], DataLand["Erro PY"], fmt='bx', label= "Pontos calculados")
plt.xlabel("Tempo (s)")
plt.ylabel(u"Posição em Y")
plt.ylim([-10, 8])
plt.xlim([0, 9])
plt.legend()
plt.grid()
plt.show()

# Analisando dados de Robo 1

In [ ]:
plt.errorbar(
    DataRob1["Tempo"][DataRob1.Time == 0],
    DataRob1["Posição X"][DataRob1.Time == 0],
    DataRob1["Erro PX"][DataRob1.Time == 0],
    fmt='kx',
    label= u"Robô indeterminado"
)
if max(DataRob1.Time[DataRob1.Time != 0]) == -1:
    plt.errorbar(
        DataRob1["Tempo"][DataRob1.Time != 0],
        DataRob1["Posição X"][DataRob1.Time != 0],
        DataRob1["Erro PX"][DataRob1.Time != 0],
        fmt='rx',
        label= u"Robô oponente"
    )
else:
    plt.errorbar(
        DataRob1["Tempo"][DataRob1.Time != 0],
        DataRob1["Posição X"][DataRob1.Time != 0],
        DataRob1["Erro PX"][DataRob1.Time != 0],
        fmt='cx',
        label= u"Robô aliado"
    )
    
if max(DataRob1.Time[DataRob1.Time != 0]) == -1:
    plt.plot(
        [a["time"]+0.5 for a in clistaOponente if a["tag"] == 2],
        [a["pos"][0] for a in clistaOponente if a["tag"] == 2],
        "k^",
        label="Dado indeterminado"
    )
    
    plt.plot(
        [a["time"]+0.5 for a in clistaOponente if a["tag"] != 2],
        [a["pos"][0] for a in clistaOponente if a["tag"] != 2],
        "r^",
        label="Dado oponente"
    )
else:
    plt.plot(
        [a["time"]+0.5 for a in clistaAliado if a["tag"] == 2],
        [a["pos"][0] for a in clistaAliado if a["tag"] == 2],
        "k^",
        label="Dado indeterminado"
    )
    
    plt.plot(
        [a["time"]+0.5 for a in clistaAliado if a["tag"] != 2],
        [a["pos"][0] for a in clistaAliado if a["tag"] != 2],
        "r^",
        label="Dado aliado"
    )

if max(DataRob1.Time[DataRob1.Time != 0]) == -1:
    plt.plot([i["time"]+0.5 for i in clistaOponente], [i["pos"][0] for i in clistaOponente], "g", label= "Valor real")
else:
    plt.plot([i["time"]+0.5 for i in clistaAliado], [i["pos"][0] for i in clistaAliado], "g", label= "Valor real")
    
plt.xlabel("Tempo (s)")
plt.ylabel(u"Posição em X")
plt.ylim([-40, 10])
plt.xlim([0, 9])
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.errorbar(
    DataRob1["Tempo"][DataRob1.Time == 0],
    DataRob1["Posição Y"][DataRob1.Time == 0],
    DataRob1["Erro PY"][DataRob1.Time == 0],
    fmt='kx',
    label= u"Robô indeterminado"
)
if max(DataRob1.Time[DataRob1.Time != 0]) == -1:
    plt.errorbar(
        DataRob1["Tempo"][DataRob1.Time != 0],
        DataRob1["Posição Y"][DataRob1.Time != 0],
        DataRob1["Erro PY"][DataRob1.Time != 0],
        fmt='rx',
        label= u"Robô oponente"
    )
else:
    plt.errorbar(
        DataRob1["Tempo"][DataRob1.Time != 0],
        DataRob1["Posição Y"][DataRob1.Time != 0],
        DataRob1["Erro PY"][DataRob1.Time != 0],
        fmt='cx',
        label= u"Robô aliado"
    )
    
if max(DataRob1.Time[DataRob1.Time != 0]) == -1:
    plt.plot(
        [a["time"]+0.5 for a in clistaOponente if a["tag"] == 2],
        [a["pos"][1] for a in clistaOponente if a["tag"] == 2],
        "k^",
        label="Dado indeterminado"
    )
    
    plt.plot(
        [a["time"]+0.5 for a in clistaOponente if a["tag"] != 2],
        [a["pos"][1] for a in clistaOponente if a["tag"] != 2],
        "r^",
        label="Dado oponente"
    )
else:
    plt.plot(
        [a["time"]+0.5 for a in clistaAliado if a["tag"] == 2],
        [a["pos"][1] for a in clistaAliado if a["tag"] == 2],
        "k^",
        label="Dado indeterminado"
    )
    
    plt.plot(
        [a["time"]+0.5 for a in clistaAliado if a["tag"] != 2],
        [a["pos"][1] for a in clistaAliado if a["tag"] != 2],
        "r^",
        label="Dado aliado"
    )

if max(DataRob1.Time[DataRob1.Time != 0]) == -1:
    plt.plot([i["time"]+0.5 for i in clistaOponente], [i["pos"][1] for i in clistaOponente], "g", label= "Valor real")
else:
    plt.plot([i["time"]+0.5 for i in clistaAliado], [i["pos"][1] for i in clistaAliado], "g", label= "Valor real")
    
plt.xlabel("Tempo (s)")
plt.ylabel(u"Posição em Y")
plt.ylim([-40, 10])
plt.xlim([0, 9])
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.errorbar(
    DataRob2["Tempo"][DataRob2.Time == 0],
    DataRob2["Posição X"][DataRob2.Time == 0],
    DataRob2["Erro PX"][DataRob2.Time == 0],
    fmt='kx',
    label= u"Robô indeterminado"
)
if max(DataRob2.Time[DataRob2.Time != 0]) == -1:
    plt.errorbar(
        DataRob2["Tempo"][DataRob2.Time != 0],
        DataRob2["Posição X"][DataRob2.Time != 0],
        DataRob2["Erro PX"][DataRob2.Time != 0],
        fmt='rx',
        label= u"Robô oponente"
    )
else:
    plt.errorbar(
        DataRob2["Tempo"][DataRob2.Time != 0],
        DataRob2["Posição X"][DataRob2.Time != 0],
        DataRob2["Erro PX"][DataRob2.Time != 0],
        fmt='cx',
        label= u"Robô aliado"
    )
    
if max(DataRob2.Time[DataRob2.Time != 0]) == -1:
    plt.plot(
        [a["time"]+0.5 for a in clistaOponente if a["tag"] == 2],
        [a["pos"][0] for a in clistaOponente if a["tag"] == 2],
        "k^",
        label="Dado indeterminado"
    )
    
    plt.plot(
        [a["time"]+0.5 for a in clistaOponente if a["tag"] != 2],
        [a["pos"][0] for a in clistaOponente if a["tag"] != 2],
        "r^",
        label="Dado oponente"
    )
else:
    plt.plot(
        [a["time"]+0.5 for a in clistaAliado if a["tag"] == 2],
        [a["pos"][0] for a in clistaAliado if a["tag"] == 2],
        "k^",
        label="Dado indeterminado"
    )
    
    plt.plot(
        [a["time"]+0.5 for a in clistaAliado if a["tag"] != 2],
        [a["pos"][0] for a in clistaAliado if a["tag"] != 2],
        "r^",
        label="Dado aliado"
    )

if max(DataRob2.Time[DataRob2.Time != 0]) == -1:
    plt.plot([i["time"]+0.5 for i in clistaOponente], [i["pos"][0] for i in clistaOponente], "g", label= "Valor real")
else:
    plt.plot([i["time"]+0.5 for i in clistaAliado], [i["pos"][0] for i in clistaAliado], "g", label= "Valor real")
    
plt.xlabel("Tempo (s)")
plt.ylabel(u"Posição em X")
plt.ylim([-40, 10])
plt.xlim([0, 9])
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.errorbar(
    DataRob2["Tempo"][DataRob2.Time == 0],
    DataRob2["Posição Y"][DataRob2.Time == 0],
    DataRob2["Erro PY"][DataRob2.Time == 0],
    fmt='kx',
    label= u"Robô indeterminado"
)
if max(DataRob2.Time[DataRob2.Time != 0]) == -1:
    plt.errorbar(
        DataRob2["Tempo"][DataRob2.Time != 0],
        DataRob2["Posição Y"][DataRob2.Time != 0],
        DataRob2["Erro PY"][DataRob2.Time != 0],
        fmt='rx',
        label= u"Robô oponente"
    )
else:
    plt.errorbar(
        DataRob2["Tempo"][DataRob2.Time != 0],
        DataRob2["Posição Y"][DataRob2.Time != 0],
        DataRob2["Erro PY"][DataRob2.Time != 0],
        fmt='cx',
        label= u"Robô aliado"
    )
    
if max(DataRob2.Time[DataRob2.Time != 0]) == -1:
    plt.plot(
        [a["time"]+0.5 for a in clistaOponente if a["tag"] == 2],
        [a["pos"][1] for a in clistaOponente if a["tag"] == 2],
        "k^",
        label="Dado indeterminado"
    )
    
    plt.plot(
        [a["time"]+0.5 for a in clistaOponente if a["tag"] != 2],
        [a["pos"][1] for a in clistaOponente if a["tag"] != 2],
        "r^",
        label="Dado oponente"
    )
else:
    plt.plot(
        [a["time"]+0.5 for a in clistaAliado if a["tag"] == 2],
        [a["pos"][1] for a in clistaAliado if a["tag"] == 2],
        "k^",
        label="Dado indeterminado"
    )
    
    plt.plot(
        [a["time"]+0.5 for a in clistaAliado if a["tag"] != 2],
        [a["pos"][1] for a in clistaAliado if a["tag"] != 2],
        "r^",
        label="Dado aliado"
    )

if max(DataRob2.Time[DataRob2.Time != 0]) == -1:
    plt.plot([i["time"]+0.5 for i in clistaOponente], [i["pos"][1] for i in clistaOponente], "g", label= "Valor real")
else:
    plt.plot([i["time"]+0.5 for i in clistaAliado], [i["pos"][1] for i in clistaAliado], "g", label= "Valor real")
    
plt.xlabel("Tempo (s)")
plt.ylabel(u"Posição em Y")
plt.ylim([-40, 10])
plt.xlim([0, 9])
plt.legend()
plt.grid()
plt.show()